# Import các thư viện cần thiết


In [ ]:
import warnings
import os
from tqdm import tqdm
from typing import List
import pandas as pd
from unidecode import unidecode
import plotly.express as px
import unicodedata
import math
import json
import numpy as np
os.chdir("/Users/vophuhan/Everything/University/Năm 4/Semester 2/Ứng dụng Phân tích dữ liệu thông minh/21KHDL-TikTok-Analytics")


# from langdetect import detect
# from langdetect import DetectorFactory

warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
# Hiển thị 6 chữ số thập phân
pd.set_option('display.float_format', '{:.6f}'.format)

# 1. Tiền xử lý dữ liệu thống kê từ video


Đọc dữ liệu thống kê video thành DataFrame.


In [255]:
file_path = "data/final_raw_videos.csv"

In [256]:
numeric_columns = [
    "authorStats.diggCount",
    "authorStats.followerCount",
    "authorStats.followingCount",
    "authorStats.friendCount",
    "authorStats.heart",
    "authorStats.heartCount",
    "authorStats.videoCount",
    "stats.collectCount",
    "stats.commentCount",
    "stats.playCount",
    "stats.shareCount",
    "statsV2.collectCount",
    "statsV2.commentCount",
    "statsV2.diggCount",
    "statsV2.playCount",
    "statsV2.repostCount",
    "statsV2.shareCount"
]

In [257]:
# Lấy tất cả các cột trong file
sample_df = pd.read_csv(file_path, nrows=5)  # Đọc 5 dòng đầu để lấy tên cột
all_columns = sample_df.columns.tolist()

# Định nghĩa dtype
dtype_dict = {
    col: "float64" if col in numeric_columns else "str" for col in all_columns}

In [258]:
video_info_df = pd.read_csv(file_path, low_memory=False, dtype=dtype_dict)
video_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71260 entries, 0 to 71259
Columns: 174 entries, stats.diggCount to duetInfo.duetFromId
dtypes: float64(17), object(157)
memory usage: 94.6+ MB


Loại bỏ các cột có tỷ lệ thiếu dữ liệu lớn hơn 50%


In [259]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display first 5 rows
missing_ratio.head()

music.coverLarge                             1.000000
user.commerceUserInfo.downLoadLink.android   1.000000
video.encodeUserTag                          1.000000
poi.country                                  1.000000
poi.fatherPoiId                              1.000000
dtype: float64

In [260]:
# Remove columns with missing ratios greater than 0.50
video_info_df = video_info_df.dropna(
    axis='columns', thresh=0.50 * len(video_info_df)
)

In [261]:
# Calculate the missing ratio
missing_ratio = video_info_df.isna().sum() / len(video_info_df)

# Sort the missing ratio in descending order
missing_ratio = missing_ratio.sort_values(ascending=False)

# Display the missing ratio
for column, ratio in missing_ratio[:5].items():
    print(f"{column:50}:{ratio:8.2%}")

video.claInfo.originalLanguageInfo.languageID     :  30.39%
video.claInfo.originalLanguageInfo.languageCode   :  30.39%
video.claInfo.captionsType                        :  30.39%
video.claInfo.originalLanguageInfo.canTranslateRealTimeNoCheck:  30.39%
video.claInfo.originalLanguageInfo.language       :  30.39%


In [262]:
video_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71260 entries, 0 to 71259
Data columns (total 87 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   stats.diggCount                                                 71260 non-null  object 
 1   CategoryType                                                    70996 non-null  object 
 2   author.commentSetting                                           70996 non-null  object 
 3   author.downloadSetting                                          70996 non-null  object 
 4   author.duetSetting                                              70996 non-null  object 
 5   author.ftc                                                      70996 non-null  object 
 6   author.id                                                       70996 non-null  object 
 7   author.isADVirtual                               

Vì các cột có tên bắt đầu với `stats.*` chứa cùng thông tin với các cột có tên bắt đầu với `statsV2.*`, nhưng không có thông tin về `repostCount` như `statsV2.*`. Nên ta sẽ loại bỏ các cột có tên bắt đầu với `stats.*`.


In [263]:
# Remove columns starting with "stats."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
     if not column.startswith("stats.")]
]

# video_info_df.info()

Xóa các cột bắt đầu với `video.claInfo.originalLanguageInfo.*` vì chúng chứa thông tin không cần thiết.


In [264]:
# Remove columns starting with "video.claInfo.originalLanguageInfo."
video_info_df = video_info_df[
    [column for column in video_info_df.columns
     if not column.startswith("video.claInfo.originalLanguageInfo.")]
]

# video_info_df.info()

Chuyển timestamp sang datetime và chuyển sang giờ VN.


In [265]:
# Chuyển timestamp sang dạng datetime UTC
video_info_df["createTime"] = pd.to_datetime(
    video_info_df["createTime"], unit="s", utc=True)

# Chuyển sang múi giờ Việt Nam (UTC+7)
video_info_df["createTime"] = video_info_df["createTime"].dt.tz_convert(
    "Asia/Ho_Chi_Minh")

Tạo một cột chứa danh sách các `hashtag` được trích xuất từ mô tả video. Và tính số lượng hashtag trong mỗi video.


In [266]:
# Replace missing values in "desc" column with an empty string
video_info_df["desc"] = video_info_df["desc"].fillna("")
video_info_df["desc"] = video_info_df["desc"].astype(str)
video_info_df["desc"] = video_info_df["desc"].str.strip()

# Create a new column for the hashtags
# and the number of hashtags in each video
video_info_df["hashtags"] = [""] * len(video_info_df)
video_info_df["num_hashtags"] = [0] * len(video_info_df)

# Extract hashtags from the "desc" column
# and Get the number of hashtags in each video
for index in tqdm(range(len(video_info_df))):
    # Get the description of the video
    description = video_info_df.loc[index, "desc"].strip().lower()

    if description:
        # # Remove emojis
        # description = description.encode('ascii', 'ignore').decode('ascii')
        # Chuyển tiếng Việt có dấu thành không dấu
        description = unidecode(description)

        # Add a space before all "#" characters
        description = description.replace("#", " #")

        # Find all strings starting with "#" and followed by a word
        hashtags = [word[1:] for word in description.split()
                    if word.startswith("#")]

        # Extract hashtags from the description
        video_info_df.loc[index, "hashtags"] = ",".join(hashtags).strip()

        # Get the number of hashtags
        video_info_df.loc[index, "num_hashtags"] = len(hashtags)
    else:
        video_info_df.loc[index, "hashtags"] = ""
        video_info_df.loc[index, "num_hashtags"] = 0

100%|██████████| 71260/71260 [00:10<00:00, 6802.95it/s]


In [267]:
# video_info_df["hashtags"] = video_info_df["hashtags"].apply(
#     lambda x: x.split(",") if x else [])

In [268]:
# Extract text (object) columns
text_df = video_info_df.select_dtypes('object')
unique_counts = text_df.nunique()
one_value_cols = list(unique_counts[unique_counts == 1].index)
print(f"One value columns: {one_value_cols}")
video_info_df = video_info_df.drop(columns=one_value_cols)
video_info_df.select_dtypes('object').columns

One value columns: ['author.commentSetting', 'author.ftc', 'author.isADVirtual', 'author.isEmbedBanned', 'author.privateAccount', 'author.relation', 'author.secret', 'collected', 'digged', 'duetDisplay', 'forFriend', 'item_control.can_repost', 'music.private', 'officalItem', 'originalItem', 'privateItem', 'secret', 'shareEnabled', 'stitchDisplay', 'video.claInfo.captionsType', 'video.format', 'duetEnabled', 'stitchEnabled']


Index(['CategoryType', 'author.downloadSetting', 'author.duetSetting',
       'author.id', 'author.nickname', 'author.openFavorite', 'author.secUid',
       'author.signature', 'author.stitchSetting', 'author.uniqueId',
       'author.verified', 'desc', 'diversificationId', 'id', 'isAd',
       'itemCommentStatus', 'music.authorName', 'music.duration', 'music.id',
       'music.isCopyrighted', 'music.original', 'music.title', 'textLanguage',
       'textTranslatable', 'video.VQScore', 'video.bitrate',
       'video.claInfo.enableAutoCaption', 'video.claInfo.hasOriginalAudio',
       'video.codecType', 'video.definition', 'video.duration',
       'video.encodedType', 'video.height', 'video.id', 'video.ratio',
       'video.videoID', 'video.videoQuality', 'video.volumeInfo.Loudness',
       'video.volumeInfo.Peak', 'video.width', 'collectTime', 'hashtags'],
      dtype='object')

- `video.definition` có cùng thông tin với `video.ratio`, `video.definition` => xoá.
- `video.videoID` dạng mã code => xoá.


In [269]:
# cols_to_delete = ['video.definition', 'video.videoID']

In [270]:
# num_df = video_info_df.select_dtypes('number')
# num_df.columns

### Chọn ra một số cột để kiểm tra thông số.


In [271]:
# selected_cols = {
#     'id': 'id',
#     'author.uniqueId': 'username',
#     "statsV2.playCount": "views",  # Lượt xem video
#     "statsV2.diggCount": "likes",  # Lượt thích video
#     "statsV2.commentCount": "comments",  # Lượt bình luận video
#     "statsV2.shareCount": "shares",  # Lượt chia sẻ video
#     'statsV2.collectCount': "collects",
#     'authorStats.diggCount': "user.likes",
#     'authorStats.followerCount': "user.followers",
#     'authorStats.followingCount': "user.followings",
#     'authorStats.friendCount': "user.friends",
#     'authorStats.heartCount': "user.hearts",
#     'authorStats.videoCount': "user.videos",
#     'hashtags': 'hashtags',
#     'createTime': 'createTime',
#     'desc': 'desc',
#     'textLanguage': 'textLang'
# }
# stats_df = video_info_df[list(selected_cols.keys())].rename(
#     columns=selected_cols)
# print(stats_df.shape)
# stats_df.sample(5)

Drop các dòng có thiếu hơn 50% số cột.


In [272]:
# stats_df["missing_count"] = stats_df.isnull().sum(axis=1)
# # Xóa các dòng có hơn 50% giá trị bị thiếu
# threshold = stats_df.shape[1] * 0.5

# # Lọc bỏ các dòng có số lượng giá trị thiếu vượt quá ngưỡng
# stats_df = stats_df[stats_df["missing_count"] <= threshold].drop(
#     columns=["missing_count"])
# stats_df.shape

# 2. Video trending


In [273]:
video_info_df['year'] = video_info_df['createTime'].dt.isocalendar().year
video_info_df['week'] = video_info_df['createTime'].dt.isocalendar().week
video_info_df['engagement_rate'] = (video_info_df['statsV2.collectCount'] + video_info_df['statsV2.commentCount'] +
                                    video_info_df['statsV2.shareCount'] + video_info_df['statsV2.diggCount'] +
                                    video_info_df['statsV2.repostCount']) / video_info_df['statsV2.playCount']

In [276]:
stats_cols = ['statsV2.playCount',
              'statsV2.diggCount',
              'statsV2.commentCount',
              'statsV2.shareCount',
              'statsV2.collectCount',
              'statsV2.repostCount',
              'engagement_rate'
              ]

In [284]:
def compute_weekly_score_and_rank(df):
    """
    Tính điểm tổng hợp theo tuần và xếp hạng video theo từng tuần, 
    đồng thời thêm xếp hạng riêng cho từng chỉ số.
    """

    df_copy = df.copy()

    # Trọng số của từng chỉ số
    columns_weights = {
        'statsV2.playCount': 0.40,
        "engagement_rate": 0.1,
        "statsV2.shareCount": 0.15,
        "statsV2.commentCount": 0.10,
        "statsV2.diggCount": 0.25
    }

    # Danh sách các cột cần xếp hạng riêng
    rank_columns = list(columns_weights.keys()) + \
        ["statsV2.collectCount", "statsV2.repostCount"]

    # Xử lý NaN & giá trị âm
    for col in columns_weights.keys():
        df_copy[col] = df_copy[col].fillna(0).clip(lower=0)

    # Tính điểm và xếp hạng trong từng tuần
    df_copy = df_copy.groupby(["year", "week"], group_keys=False).apply(
        lambda group: _compute_score_and_ranking_per_week(
            group, columns_weights, rank_columns)
    )

    return df_copy


def _compute_score_and_ranking_per_week(group, columns_weights, rank_columns):
    """
    Tính điểm và xếp hạng trong từng tuần riêng lẻ.
    """
    # Chuẩn hóa theo max trong tuần
    max_values = {col: group[col].max() or 1 for col in columns_weights.keys()}

    for col in columns_weights.keys():
        group[f"norm_{col}"] = group[col] / max_values[col]

    # Tính điểm tổng hợp
    group["weekly_score"] = sum(
        columns_weights[col] * group[f"norm_{col}"] * 100
        for col in columns_weights.keys()
    )

    # Xếp hạng tổng điểm theo tuần
    group["weekly_score_rank"] = group["weekly_score"].rank(
        method="dense", ascending=False).astype("Int64")

    # Xếp hạng riêng cho từng chỉ số trong tuần
    for col in rank_columns:
        group[f"{col}_rank"] = group[col].rank(
            method="dense", ascending=False).astype("Int64")

    # Xóa cột chuẩn hóa sau khi tính toán xong
    norm_columns = [f"norm_{col}" for col in columns_weights.keys()]
    group = group.drop(columns=norm_columns)
    return group


# Gọi hàm trên DataFrame video_info_df
# selected_df = video_info_df[['year', 'week',
#                              'author.uniqueId', 'id', ]+stats_cols]
selected_df = video_info_df
selected_df = compute_weekly_score_and_rank(selected_df)

# Hiển thị số nguyên đầy đủ
with pd.option_context('display.float_format', '{:.3f}'.format):
    display(selected_df[selected_df['year'] == 2025].sort_values(
        ['year', 'week', 'weekly_score_rank'], ascending=True).head(5))

,CategoryType,author.downloadSetting,author.duetSetting,author.id,author.nickname,author.openFavorite,author.secUid,author.signature,author.stitchSetting,author.uniqueId,author.verified,authorStats.diggCount,authorStats.followerCount,authorStats.followingCount,authorStats.friendCount,authorStats.heart,authorStats.heartCount,authorStats.videoCount,createTime,desc,diversificationId,id,isAd,itemCommentStatus,music.authorName,music.duration,music.id,music.isCopyrighted,music.original,music.title,statsV2.collectCount,statsV2.commentCount,statsV2.diggCount,statsV2.playCount,statsV2.repostCount,statsV2.shareCount,textLanguage,textTranslatable,video.VQScore,video.bitrate,video.claInfo.enableAutoCaption,video.claInfo.hasOriginalAudio,video.codecType,video.definition,video.duration,video.encodedType,video.height,video.id,video.ratio,video.videoID,video.videoQuality,video.volumeInfo.Loudness,video.volumeInfo.Peak,video.width,collectTime,hashtags,num_hashtags,year,week,engagement_rate,weekly_score,weekly_score_rank,statsV2.playCount_rank,engagement_rate_rank,statsV2.shareCount_rank,statsV2.commentCount_rank,statsV2.diggCount_rank,statsV2.collectCount_rank,statsV2.repostCount_rank
64493,111.0,3.0,0.0,7074548497097344026,Panda Chân Lý,False,MS4wLjABAAAAYQK0lSO-lNEGWsCnhXZrDCAhe6LYfipzbs...,👉🏻KHÔNG NHẬN BOOKING QUÁN ĂN,0.0,hoanganhdeptrailam,False,9725.000,912000.000,52.000,0.000,38500000.000,38500000.000,458.000,2025-01-01 13:00:00+07:00,Ốc vòi voi @panda.review0chuyen #ancungtiktok ...,10041.0,7454790808231890194,False,0.0,Panda Chân Lý,303.0,7454791839204608785,False,True,nhạc nền - Panda Chân Lý,13864.000,12700.000,226300.000,5300000.000,0.000,106500.000,vi,True,69.68,1168347.0,True,False,h264,540p,303.0,normal,1024.0,7454790808231890194,540p,v10025g50000ctqbgonog65oaji91c30,normal,-15.0,1.0,576.0,1742377991.0,"ancungtiktok,tiktokgiaitri,hoanganhpanda",3,2025,1,0.068,50.432,1,10,108,1,1,4,13,1
15859,111.0,3.0,0.0,6869980717103842305,Tóc Vàng hoe,False,MS4wLjABAAAAToh4HJZ0bodghQPED1oi7FY43d9OcjNOWB...,Đồ makeup và quần áo Vàng mặc trong clip ở đây...,0.0,tocvang.hoe,False,968.000,2400000.000,5.000,0.000,44800000.000,44800000.000,404.000,2025-01-03 16:58:22+07:00,Mukbang blindbox uống trà Lipton nhãn vàng cùn...,10041.0,7455626418827824402,True,0.0,Tóc Vàng hoe,108.0,7455626546133437201,False,True,nhạc nền - Tóc Vàng hoe,4259.000,412.000,71600.000,20400000.000,0.000,156.000,vi,True,74.5,1088006.0,True,True,h264,540p,108.0,normal,1024.0,7455626418827824402,540p,v14044g50000ctrr8jfog65sp9oiikn0,normal,-15.1,1.0,576.0,1742378213.0,"tocvanghoe,mukbang,lipton,blindbox,ancungtikto...",6,2025,1,0.004,44.818,2,1,1129,435,56,32,69,1
23117,111.0,0.0,0.0,7387796761811059718,Hoàng Anh Vào Bếp,False,MS4wLjABAAAAX2_LYZj8q-h3CyiMHmruYr_5eMONMcMWLu...,Forwork booking\n📩 nguyenhoanganh.banker@gmail...,0.0,hoanganhvaobep,False,269.000,25400.000,91.000,0.000,264700.000,264700.000,67.000,2024-12-31 18:05:08+07:00,Phần 15/100: Trứng Rim Mắm Nay là ngày cuối củ...,10040.0,7454530370634059015,False,0.0,"𝓜𝓾𝓼𝓲𝓬 𝓬𝓱𝓲𝓵𝓵,🎧",300.0,7072602716777859866,False,True,"nhạc nền - 𝓜𝓾𝓼𝓲𝓬 𝓬𝓱𝓲𝓵𝓵,🎧",81266.000,3266.000,209000.000,11900000.000,0.000,38600.000,vi,True,70.88,1762722.0,True,True,h264,540p,53.0,normal,1024.0,7454530370634059015,540p,v14044g50000ctpsq2fog65h8tt6h9ag,normal,-26.3,0.35075,576.0,1742377990.0,"nauancungtiktok,learnontiktok,ancungtiktok,vir...",9,2025,1,0.028,44.670,3,3,577,3,3,6,1,1
67430,111.0,3.0,0.0,7001315265619936258,Ớt review,False,MS4wLjABAAAAxSbPNwWRrFKXH-9KqXPiSBxDXhcxnBtvwh...,Liên hệ công việc : 0392415851 Ms Loan\nCẩm na...,0.0,ot.revieww,False,21800.000,618300.000,353.000,0.000,11300000.000,11300000.000,530.000,2025-01-04 17:27:32+07:00,Ăn cùng người lạ THPT An Dương p60 #otreview #...,10085.0,7456005004818681106,False,0.0,文武贝,170.0,7048037465826871327,False,False,Happy Day,23379.000,2445.000,405800.000,7800000.000,0.000,9727.000,vi,True,67.62,1064233.0,True,True,h264,540p,137.0,normal,1024.0,7456005004818681106,540p,v14044g50000ctsgpn7og65n77e2

In [ ]:
selected_df.to_csv("data/videos_with_weekly_rank.csv")

# 3. Lọc lại danh sách user


In [278]:
print("Number of users: ", stats_df['username'].nunique())

NameError: name 'stats_df' is not defined

Lọc các video đăng trong năm 2025 (loại bỏ người dùng không hoạt động).


In [ ]:
# time_thres = pd.to_datetime("2025-01-01").tz_localize("Asia/Ho_Chi_Minh")
# filt_stats_df = stats_df[stats_df['createTime'] >= time_thres]
filt_stats_df = stats_df

print("Number of users: ", filt_stats_df['username'].nunique())

### Các thông số thống kê theo từng user về lượt xem và độ tương tác.


In [ ]:
# Lọc lại các cột quan trọng liên quan đến user và các chỉ số tương tác của video
df_selected = filt_stats_df[[
    # "author.id",  # ID của user
    "username",  # Tên hiển thị của user
    "views",  # Lượt xem video
    "likes",  # Lượt thích video
    "comments",  # Lượt bình luận video
    "shares",  # Lượt chia sẻ video
    'collects',
    'user.followers',
    'user.likes'
]]

# Kiểm tra dữ liệu bị thiếu
df_selected = df_selected.dropna()

# Tổng hợp dữ liệu theo user
user_stats_df_0 = df_selected.groupby(["username"]).agg(
    total_views=("views", "sum"),
    avg_views=("views", "mean"),
    total_likes=("likes", "sum"),
    avg_likes=("likes", "mean"),
    total_comments=("comments", "sum"),
    total_shares=("shares", "sum"),
    total_collects=("collects", "sum"),
    total_videos=("views", "count"),  # Số lượng video của user
    followers=('user.followers', 'max'),
    likes=('user.likes', 'max'),
).reset_index()

user_stats_df_0 = user_stats_df_0.rename(columns={'username': 'username'})
# Tính Engagement Rate (ER) trung bình
user_stats_df_0["engagement_rate"] = (user_stats_df_0["total_likes"] + user_stats_df_0["total_comments"] +
                                      user_stats_df_0["total_shares"]+user_stats_df_0["total_collects"]) / user_stats_df_0["total_views"]
user_stats_df_0["total_engagement"] = (user_stats_df_0["total_likes"] + user_stats_df_0["total_comments"] +
                                       user_stats_df_0["total_shares"]+user_stats_df_0["total_collects"])
# user_stats_df_0["avg_engagement"] = user_stats_df_0["total_engagement"] / \
#     user_stats_df_0["total_videos"]
# Loại bỏ những user có tổng lượt xem bằng 0 (tránh lỗi chia cho 0)
user_stats_df_0 = user_stats_df_0[user_stats_df_0["total_views"] > 0]
display(user_stats_df_0.sample(5))
# display(user_stats_df_0.describe())

### Phần trăm video có caption bằng tiếng Việt


In [ ]:
# Tổng số video theo từng user
total_videos = stats_df.groupby('username')['textLang'].count()

# Số video có textLang là 'vi'
vi_videos = stats_df[stats_df['textLang'] == 'vi'].groupby('username')[
    'textLang'].count()

# Ghép hai bảng, thay NaN bằng 0 (nếu user không có video tiếng Việt)
percent_vi = (vi_videos / total_videos).fillna(0).reset_index()

# Đổi tên cột
percent_vi.columns = ['username', 'vi_video_percentage']
percent_vi.sort_values('vi_video_percentage', ascending=False)

### Khoảng thời gian giữa 2 bài đăng của user mà 75% bài đăng thoả ngưỡng đó.

Tức là 75% các video của có khoảng cách giữa các lần đăng dưới ngưỡng này.


In [ ]:
select_df = filt_stats_df
select_df = select_df.sort_values(by=["username", "createTime"])

# Tính khoảng cách thời gian giữa các video liên tiếp của mỗi user
select_df["time_diff"] = select_df.groupby("username")["createTime"].diff()

# Xử lý trường hợp user chỉ có 1 video, không tính được diff
select_df["time_diff"] = select_df["time_diff"].fillna(pd.Timedelta(seconds=0))

# Tính trung bình khoảng cách giữa các video cho mỗi user
frequency_df = select_df.groupby("username")["time_diff"].quantile(
    0.75).dt.total_seconds() / 3600 / 24  # Đổi ra ngày

# Nếu chỉ có 1 video, gán giá trị 'inf' cho 'days_between_videos'
frequency_df = frequency_df.reset_index(name="days_between_videos")

# Xử lý trường hợp user có 1 video
single_video_users = frequency_df[frequency_df["days_between_videos"] == 0]
frequency_df.loc[single_video_users.index, "days_between_videos"] = float(
    'inf')  # Gán giá trị 'inf' cho những người chỉ có 1 video
frequency_df['frequency'] = 1 / frequency_df['days_between_videos']

# Kiểm tra kết quả
frequency_df.head()

### Tính phần trăm cái bài đăng có liên quan đến ẩm thực (dựa trên hashtag)


Danh sách các hashtag, số video tương ứng và số user sử dụng hashtag đó.


In [ ]:
# Tách hashtag thành từng dòng riêng biệt
exploded_df = stats_df.explode("hashtags")

# Đếm số lượng video chứa mỗi hashtag
hashtag_video_counts = exploded_df["hashtags"].value_counts().reset_index()
hashtag_video_counts.columns = ["hashtags", "videos_count"]

# Đếm số tài khoản sử dụng mỗi hashtag (số user duy nhất)
hashtag_user_counts = exploded_df.groupby(
    "hashtags")["username"].nunique().reset_index()
hashtag_user_counts.columns = ["hashtags", "users_count"]

# Gộp cả 2 thông tin vào cùng 1 DataFrame
hashtags_df = hashtag_video_counts.merge(hashtag_user_counts, on="hashtags")
hashtags_df["avg_videos_per_user"] = hashtags_df["videos_count"] / \
    hashtags_df["users_count"]
hashtags_df = hashtags_df.sort_values('users_count', ascending=False)
display(hashtags_df)
# px.bar(hashtags_df.head(50), x='users_count', y='hashtags')

Lọc ra các hashtag có liên quan đến ẩm thực.


In [ ]:
food_keywords = [
    "food", "eat", "drink", "chef", "cook", "cooking", "recipe", "baking", "buffet", "steak", "grill", "vegan",
    "anvat",  "anuong",  "angi",  "anngon", "doan", "ancung", 'nauan', 'anchay',
    "douong", "nuocuong",
    "monngon", 'monchay',
    "ancom", "anvat",  "moncom", "buacom", 'comnha', 'buacom', 'comtam', 'comchay',
    "hutieu", "banhmi", "haisan", 'micay', 'mituongden', 'mycay', 'mytuongden',
    "trasua", "nuoc", "caphe", "sinhto", "ruou", 'latte', 'cacao', 'caffe', 'bun', 'banh',
    "amthuc",  "nhahang", "thucdon", "dacsan", "mukbang",
]

# food_keywords = [
#     "food", "eat", "drink", "chef", "cook", "cooking", "recipe", "baking", "buffet", "steak", "grill", "vegan",
#     "amthuc", "an",  "nhahang", "mon", "ngon", "uong", "thucdon", "dacsan", "tiem",
#     # "ancom", "anvat",  "moncom", "buacom", 'comnha', 'buacom',
#     "hutieu", "banhmi", "haisan", "com", 'bun', 'canh',
#     "tra", "sua", "nuoc", "caphe", "sinhto", "ruou", 'latte', 'cacao', 'caffe',
#     'banh',
# ]

food_related_hashtags_df = hashtags_df[hashtags_df["hashtags"].apply(
    lambda x: any(kw in x for kw in food_keywords))]
food_related_hashtags_df = food_related_hashtags_df.sort_values(
    'users_count', ascending=False, ignore_index=True)

# with pd.option_context("display.max_rows", None):
display(food_related_hashtags_df)

In [ ]:
px.bar(food_related_hashtags_df.head(30),
       y='hashtags', x='users_count', height=700)

In [ ]:
# food_related_hashtags_df.to_csv("hashtag_list.csv", index=False)

Tính phần trăm các video có hashtag về ẩm thực của từng user.


In [ ]:
# Danh sách các hashtag liên quan đến ẩm thực
# food_related_hashtags = {
#     "reviewanngon", "ancungtiktok", "foodreview", "monngon", "anvat",
#     "foodtour", "eatclean", "buacomgiadinh", "amthuc", "diadiemanuong",
#     "reviewanuongsaigon", "anvatsaigon", "saigonfood"
# }

# Bước 1: Lấy ra 100 hashtag ẩm thực có nhiều video nhất
food_related_hashtags = food_related_hashtags_df.head(50)[
    'hashtags'].to_list()
# Bước 2: Tạo cột mới để đánh dấu video có chứa ít nhất một hashtag liên quan đến ẩm thực
filt_stats_df["is_food_related"] = filt_stats_df["hashtags"].apply(
    lambda tags: any(tag in food_related_hashtags for tag in tags)
)

# Bước 3: Tính tổng số video & số video liên quan đến ẩm thực của mỗi user
food_user_df = filt_stats_df.groupby("username").agg(
    total_videos=("username", "count"),
    food_related_videos=("is_food_related", "sum")
).reset_index()

# Bước 4: Tính phần trăm số video ẩm thực
food_user_df["food_video_percentage"] = (
    food_user_df["food_related_videos"] / food_user_df["total_videos"])

food_user_df = food_user_df.rename(columns={"username": "username"})
food_user_df.head()

Merge với `user_stats_df_0`.


In [ ]:
user_stats_df = user_stats_df_0.merge(percent_vi, on='username').merge(frequency_df, on="username").merge(
    food_user_df, on=['username', 'total_videos'])
user_stats_df = user_stats_df[user_stats_df['frequency'] != 0]
user_stats_df.head()

display(user_stats_df.sample(5))

### Lọc user theo các ngưỡng


#### Visualization


In [ ]:
px.histogram(user_stats_df, x='vi_video_percentage',
             title="Phân phối User theo Phần trăm video có caption tiếng Việt")

In [ ]:
px.histogram(user_stats_df, x='food_video_percentage',
             title="Phân phối User theo Phần trăm video liên quan đến Ẩm thực", nbins=20)

In [ ]:
px.histogram(user_stats_df, x='days_between_videos',
             title="Phân phối User theo Khoảng thời gian giữa các bài đăng")

In [ ]:
px.histogram(user_stats_df, x='engagement_rate',
             title="Phân phối User theo Tỷ lệ tương tác")

In [ ]:
px.histogram(user_stats_df, x='avg_views',
             title="Phân phối User theo Lượt xem trung bình")

In [ ]:
px.histogram(user_stats_df, x='followers',
             title="Phân phối User theo Followers")

In [ ]:
px.histogram(user_stats_df, x='likes',
             title="Phân phối User theo Likes")

#### Lọc


In [ ]:
print(user_stats_df.shape)
user_stats_df[['username', 'avg_views', 'total_videos', 'followers',
               'engagement_rate', 'frequency',
               'food_video_percentage', 'vi_video_percentage']].describe()

In [ ]:
# Lọc lại danh sách user
# users_by_threshold = user_stats_df[
#     # (df_grouped["total_videos"] >= 4) &
#     (
#         (user_stats_df["food_video_percentage"]
#          >= min_food_perc_threshold)
#         &
#         (user_stats_df["frequency"]
#          >= min_freq_threshold)
#         &
#         (user_stats_df["engagement_rate"]
#          >= min_eng_rate_threshold)
#         &
#         (user_stats_df["avg_views"]
#          >= min_avg_views_threshold)
#     )
# ]

print(user_stats_df.shape)

# Bước 0: Lọc theo vi_video_percentage
min_vi_perc_threshold = 0.90
users_by_threshold = user_stats_df[
    user_stats_df["vi_video_percentage"] >= min_vi_perc_threshold
]
print(f"Min Vietnamese Video Percentage Threshold: {min_vi_perc_threshold}")
print(users_by_threshold.shape)

# Bước 1: Lọc theo food_video_percentage
# min_food_perc_threshold = users_by_threshold["food_video_percentage"].quantile(
#     0.6)
min_food_perc_threshold = 0.75
users_by_threshold = users_by_threshold[
    users_by_threshold["food_video_percentage"] >= min_food_perc_threshold
]
print(f"Min Food Video Percentage Threshold: {min_food_perc_threshold}")
print(users_by_threshold.shape)

# # Bước 2: Lọc tiếp theo avg_views
# min_avg_views_threshold = users_by_threshold["avg_views"].quantile(0.1)
# users_by_threshold = users_by_threshold[
#     users_by_threshold["avg_views"] >= min_avg_views_threshold
# ]
# print(f"Min Average Views Threshold: {min_avg_views_threshold}")
# print(users_by_threshold.shape)

# # Bước 3: Lọc tiếp theo engagement_rate
# min_eng_rate_threshold = users_by_threshold["engagement_rate"].quantile(0.1)
# users_by_threshold = users_by_threshold[
#     users_by_threshold["engagement_rate"] >= min_eng_rate_threshold
# ]
# print(f"Min Engagement Rate Threshold: {min_eng_rate_threshold}")
# print(users_by_threshold.shape)

# # Bước 4: Lọc cuối cùng theo frequency
# min_freq_threshold = users_by_threshold["frequency"].quantile(0.1)
# users_by_threshold = users_by_threshold[
#     users_by_threshold["frequency"] >= min_freq_threshold
# ]
# print(f"Min Frequency Threshold: {min_freq_threshold}")
# print(users_by_threshold.shape)

users_by_threshold.sort_values(by="total_videos", ascending=True)

In [ ]:
video_info_df[video_info_df['author.uniqueId'].isin(
    users_by_threshold['username'])].shape

In [ ]:
def write_list_to_file(file_path, data_list):
    try:
        with open(file_path, "w", encoding="utf-8") as file:
            for item in data_list:
                file.write(str(item) + "\n")
        print(f"Successfully wrote {len(data_list)} items to {file_path}")
    except Exception as e:
        print(f"Error writing to file: {e}")


write_list_to_file('data/lists/final_users_list.txt',
                   list(users_by_threshold['username'].unique()))

### Lọc bằng cách tính điểm


In [ ]:

def compute_score(df, columns_weights):
    """
    Tính điểm tổng hợp cho DataFrame user_stats_df với các cột và trọng số linh hoạt.

    Công thức tính điểm: 
      score = sum(weight_i * norm_column_i)
    """
    df_copy = df.copy()

    for col in columns_weights.keys():
        df_copy[col] = df_copy[col].fillna(0).clip(lower=0)

    max_values = {col: df_copy[col].max(
    ) or 1 for col in columns_weights.keys()}

    for col in columns_weights.keys():
        df_copy[f"norm_{col}"] = df_copy[col] / max_values[col]

    df_copy["score"] = sum(
        columns_weights[col] * df_copy[f"norm_{col}"]*100
        for col in columns_weights.keys()
    )

    return df_copy

In [ ]:

# Ví dụ sử dụng:
columns_weights = {
    "food_video_percentage": 0.35,
    "engagement_rate": 0.05,
    "avg_views": 0.35,
    "frequency": 0.1,
    "followers": 0.35
}
# columns_weights = {
#     "avg_views": 0.35,
#     "engagement_rate": 0.25,
#     "followers": 0.35,
#     "frequency": 0.05
# }

# Tính điểm cho DataFrame
# users_by_score = compute_score(
#     user_stats_df[user_stats_df['vi_video_percentage'] >= 0.8], columns_weights)
# users_by_score = compute_score(users_by_threshold, columns_weights)
users_by_score = compute_score(
    users_by_threshold, columns_weights)

# In kết quả
users_by_score.sort_values('score', ascending=False)

# Sau đó bạn có thể:
users_by_score = users_by_score.sort_values(
    "score", ascending=False, ignore_index=True)

uesr_by_score_top = users_by_score.head(len(users_by_threshold))
display(users_by_score[["username", "score"]].merge(user_stats_df[[
        'followers', 'username', 'avg_views', 'frequency', 'engagement_rate']]))

In [ ]:
user_stats_df.columns

In [ ]:
import json


def calculate_entropy_weights(df, columns_to_maximize):
    """
    Tính trọng số (weights) cho mỗi cột trong DataFrame df dựa trên phương pháp Entropy Weighting.
    """
    # 1) Chuẩn hóa dữ liệu về [0..1]
    df_normalized = df[columns_to_maximize].copy()
    for col in columns_to_maximize:
        df_normalized[col] = (df[col] - df[col].min()) / \
            (df[col].max() - df[col].min())

    # 2) Tính entropy cho mỗi cột
    n = len(df)  # số lượng user
    p = df_normalized / df_normalized.sum(axis=0)  # xác suất
    entropy = - (p * np.log(p + 1e-9)).sum(axis=0) / np.log(n)  # tránh log(0)

    # 3) Tính trọng số (weights) cho mỗi cột
    weights = (1 - entropy) / (1 - entropy).sum()

    # Trả về DataFrame với trọng số
    weights_df = pd.DataFrame(
        weights, columns=['weight'], index=columns_to_maximize)

    return weights_df


columns_to_maximize = ['total_views', 'avg_views', 'total_videos', 'total_engagement', 'followers',
                       # 'total_likes', 'avg_likes', 'total_comments', 'total_shares', 'total_collects', 'cluster', 'days_between_videos',
                       'engagement_rate', 'frequency', 'food_related_videos', 'food_video_percentage']


# Tính trọng số cho mỗi cột
weights_df = calculate_entropy_weights(
    user_stats_df, columns_to_maximize).sort_values("weight")
display(weights_df)
# Xem kết quả
columns_weights = json.loads(weights_df.to_json())['weight']

In [ ]:
# Tính điểm cho DataFrame
df_with_scores = compute_score(user_stats_df, columns_weights)

# In kết quả
df_with_scores.sort_values('score', ascending=False)

# Sau đó bạn có thể:
user_stats_df_scored = df_with_scores.sort_values(
    "score", ascending=False)
user_stats_df_scored[["username", "score"]].head(100)  # Xem top 20

In [ ]:
# from sklearn.cluster import KMeans
# columns_to_consider = ['total_views', 'avg_views', 'total_likes', 'avg_likes',
#                        'total_comments', 'total_shares', 'total_collects',
#                        'total_videos', 'engagement_rate', 'days_between_videos',
#                        'food_related_videos', 'food_video_percentage', 'frequency']
# # Sử dụng các cột để phân nhóm (có thể chuẩn hóa dữ liệu nếu cần)
# X = user_stats_df[columns_to_consider]

# # Áp dụng KMeans (giả sử chia thành 3 nhóm)
# kmeans = KMeans(n_clusters=3)
# user_stats_df['cluster'] = kmeans.fit_predict(X)

# # Lấy các centroid của mỗi nhóm
# centroids = kmeans.cluster_centers_

# # Các cột quan trọng có thể được xác định từ các giá trị lớn trong centroids
# centroid_df = pd.DataFrame(centroids, columns=columns_to_consider)
# centroid_df

In [ ]:
# def get_pareto_front(df, columns_maximize):
#     """
#     Tìm Pareto front trong DataFrame df, coi tất cả cột trong columns_maximize
#     đều là "càng lớn càng tốt".

#     Trả về sub-DataFrame gồm các hàng không bị chi phối bởi bất kỳ hàng nào khác.
#     """
#     dominated = set()
#     n = len(df)

#     for i in range(n):
#         if i in dominated:
#             # Hàng này đã bị chi phối, không cần check nữa
#             continue

#         # Kiểm tra xem có hàng j nào chi phối i không
#         for j in range(n):
#             if i == j:
#                 continue
#             # Nếu j >= i trên toàn bộ columns,
#             # và j > i trên ít nhất 1 cột => i bị chi phối
#             if (df.iloc[j][columns_maximize] >= df.iloc[i][columns_maximize]).all() \
#                and (df.iloc[j][columns_maximize] > df.iloc[i][columns_maximize]).any():
#                 dominated.add(i)
#                 break

#     # Các hàng không bị chi phối -> Pareto front
#     indices_front = [idx for idx in range(n) if idx not in dominated]
#     return df.iloc[indices_front].copy()

In [ ]:
# # Tìm Pareto front
# pareto_df = get_pareto_front(user_stats_df, columns_to_maximize)

# print(f"Số user trên Pareto front: {len(pareto_df)}")
# pareto_df.head()

In [ ]:
# pareto_df.columns